<a href="https://colab.research.google.com/github/seimwiwa/playground_python/blob/main/20231216_podcast_analysis_asr.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
#@title Install
%%capture

!pip install pydub

In [29]:
#@title Import
%%capture

import torch, json, os
from google.colab import drive
from pydub import AudioSegment
from transformers import AutoModelForSpeechSeq2Seq, AutoProcessor, pipeline
import pandas as pd

In [5]:
#@title Conntect Google Drive
drive.mount("/content/drive")

Mounted at /content/drive


In [16]:
#@title List Aplited Audio
path = "/content/drive/MyDrive/(07) Colab/20231216_podcast_analysis/splited_audio/"
files = os.listdir(path)

In [6]:
#@title Prepare ASR Model
%%capture

name = "openai/whisper-large-v3"
torch_dtype = torch.float16
device = "cuda:0"

model = AutoModelForSpeechSeq2Seq.from_pretrained(name, torch_dtype = torch_dtype)
model.to(device)

processor = AutoProcessor.from_pretrained(name)

pipe = pipeline("automatic-speech-recognition", model = model, tokenizer = processor.tokenizer,
                feature_extractor = processor.feature_extractor, max_new_tokens = 128, chunk_length_s = 30,
                batch_size = 16, return_timestamps = False, torch_dtype = torch_dtype, device = device)

In [23]:
#@title Get Scripts
%%capture

scripts = [pipe(path + i, generate_kwargs = {"language": "english"}) for i in files]

In [ ]:
#@title Clean up the scripts
data = pd.DataFrame({"file_name": files, "text": [i["text"] for i in scripts]})
data["id"] = data.file_name.str.replace(".mp3", "").astype(int)
data = data.sort_values(["id"]).drop(["file_name"], axis = 1).filter(["id", "text"]).reset_index(drop = True)

In [51]:
#@title Save The Scripts
data.to_csv("/content/drive/MyDrive/(07) Colab/20231216_podcast_analysis/scripts.csv", index=False)